In [18]:
import pandas as pd
import numpy as np

In [19]:
# Load the CSV file. Make sure 'cars_data_RAW.csv' is in your working directory.
df = pd.read_csv("cars_data_RAW.csv")

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 353 entries, 0 to 352
Data columns (total 17 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Row_ID                               353 non-null    int64  
 1   title                                353 non-null    object 
 2   model                                353 non-null    object 
 3   battery                              353 non-null    float64
 4   price-range                          353 non-null    int64  
 5   0 - 100                              353 non-null    object 
 6   Top Speed                            353 non-null    object 
 7   Range*                               353 non-null    object 
 8   Efficiency*                          353 non-null    object 
 9   Fastcharge*                          353 non-null    object 
 10  Germany_price_before_incentives      340 non-null    object 
 11  Netherlands_price_before_incenti

In [21]:
df.isnull().sum()

,0
Row_ID,0
title,0
model,0
battery,0
price-range,0
0 - 100,0
Top Speed,0
Range*,0
Efficiency*,0
Fastcharge*,0


In [22]:
# 1. Cleaning: Convert performance columns to float
performance_cols = {
    '0 - 100': ' sec',
    'Top Speed': ' km/h',
    'Range*': ' km',
    'Efficiency*': ' Wh/km',
    'Fastcharge*': ' km/h'
}

In [23]:
for col, unit in performance_cols.items():
    df[col] = df[col].astype(str).str.replace(unit, '', regex=False)
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [26]:
# 2. Cleaning: Convert price columns to float
price_cols = [
    'Germany_price_before_incentives',
    'Netherlands_price_before_incentives',
    'UK_price_after_incentives'
]

In [27]:
for col in price_cols:
    df[col] = df[col].astype(str).str.replace(r'[€£,]', '', regex=True)
    df[col] = df[col].replace(['nan', 'N/A', 'NaN'], np.nan)
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [44]:
# --- Start Cross-Currency Imputation ---

# Assumed Exchange Rates (for imputation)
# 1 EUR -> 0.85 GBP
# 1 GBP -> 1.17 EUR
# A. Impute UK Price (GBP) using German Price (EUR)
condition_uk_missing = df['UK_price_after_incentives'].isnull()
condition_de_available = df['Germany_price_before_incentives'].notnull()

In [45]:
df.loc[condition_uk_missing & condition_de_available, 'UK_price_after_incentives'] = \
    df['Germany_price_before_incentives'] * 0.85

In [46]:
# # B. Impute German Price (EUR) using UK Price (GBP)
condition_de_missing = df['Germany_price_before_incentives'].isnull()
condition_uk_available = df['UK_price_after_incentives'].notnull()

In [47]:
df.loc[condition_de_missing & condition_uk_available, 'Germany_price_before_incentives'] = \
    df['UK_price_after_incentives'] * 1.17

In [49]:
# C. Impute Netherlands Price (EUR) using German Price (EUR)
condition_nl_missing = df['Netherlands_price_before_incentives'].isnull()
condition_de_available_final = df['Germany_price_before_incentives'].notnull()
df.loc[condition_nl_missing & condition_de_available_final, 'Netherlands_price_before_incentives'] = \
    df['Germany_price_before_incentives']

In [50]:
# --- Step 4: Stage 2 Imputation (Manufacturer Median for Remaining NaNs) ---

# Impute remaining price NaNs with the MEDIAN PRICE of the same manufacturer ('title')
for col in price_cols:
    # Use groupby().transform() to fill NaNs with the group median
    # This only works if there is at least one non-NaN value for that manufacturer.
    df[col] = df[col].fillna(df.groupby('title')[col].transform('median'))

In [51]:
# For the absolute final missing price values (where a manufacturer had NO data), use the global median
for col in price_cols:
    global_median = df[col].median()
    df[col] = df[col].fillna(global_median)

In [61]:
# --- 3. Calculate Estimated US Price ---

# Define conversion rates (approximate)
EUR_TO_USD = 1.08
GBP_TO_USD = 1.25

In [62]:
# Create a temporary DataFrame of USD-converted prices
usd_estimates = pd.DataFrame({
    'USD_from_DE': df['Germany_price_before_incentives'] * EUR_TO_USD,
    'USD_from_NL': df['Netherlands_price_before_incentives'] * EUR_TO_USD,
    'USD_from_UK': df['UK_price_after_incentives'] * GBP_TO_USD
})

In [63]:
# Calculate the mean of the available USD estimates for each row
df['Estimated_US_Price'] = usd_estimates.mean(axis=1)

In [64]:
# Imputation Step A: Manufacturer Median Imputation for Estimated_US_Price
# Fill NaNs with the median price for the same car manufacturer
df['Estimated_US_Price'] = df['Estimated_US_Price'].fillna(
    df.groupby('title')['Estimated_US_Price'].transform('median')
)

# Imputation Step B: Global Median Imputation (Final guarantee for price)
global_median_usd = df['Estimated_US_Price'].median()
df['Estimated_US_Price'] = df['Estimated_US_Price'].fillna(global_median_usd)

In [65]:
# Impute Fastcharge* NaNs with the global median
median_fastcharge = df['Fastcharge*'].median()
df['Fastcharge*'] = df['Fastcharge*'].fillna(median_fastcharge)

In [66]:
# Tow_Hitch: Fill NaN with a category
df['Tow_Hitch'] = df['Tow_Hitch'].fillna('No information/Not possible')

In [67]:
# Towing_capacity_in_kg: Fill NaN with 0 and convert to integer
df['Towing_capacity_in_kg'] = df['Towing_capacity_in_kg'].fillna(0).astype(int)

In [68]:
# Display the results of the cleaning
print("\n--- Cleaned DataFrame Head ---")
print(df.head().to_markdown(index=False, numalign="left", stralign="left"))


--- Cleaned DataFrame Head ---
| Row_ID   | title   | model                         | battery   | price-range   | 0 - 100   | Top Speed   | Range*   | Efficiency*   | Fastcharge*   | Germany_price_before_incentives   | Netherlands_price_before_incentives   | UK_price_after_incentives   | Drive_Configuration   | Tow_Hitch       | Towing_capacity_in_kg   | Number_of_seats   | Estimated_US_Price   |
|:---------|:--------|:------------------------------|:----------|:--------------|:----------|:------------|:---------|:--------------|:--------------|:----------------------------------|:--------------------------------------|:----------------------------|:----------------------|:----------------|:------------------------|:------------------|:---------------------|
| 1        | Tesla   | Model 3                       | 57.5      | 104           | 6.1       | 201         | 415      | 139           | 690           | 41970                             | 41990                                 | 39

In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 353 entries, 0 to 352
Data columns (total 18 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Row_ID                               353 non-null    int64  
 1   title                                353 non-null    object 
 2   model                                353 non-null    object 
 3   battery                              353 non-null    float64
 4   price-range                          353 non-null    int64  
 5   0 - 100                              353 non-null    float64
 6   Top Speed                            353 non-null    int64  
 7   Range*                               353 non-null    int64  
 8   Efficiency*                          353 non-null    int64  
 9   Fastcharge*                          353 non-null    float64
 10  Germany_price_before_incentives      353 non-null    float64
 11  Netherlands_price_before_incenti

In [70]:
df.isnull().sum()

,0
Row_ID,0
title,0
model,0
battery,0
price-range,0
0 - 100,0
Top Speed,0
Range*,0
Efficiency*,0
Fastcharge*,0


In [71]:
# Save the final cleaned DataFrame to a new CSV file
imputed_file_name = "cars_data_CROSS_IMPUTED.csv"
df.to_csv(imputed_file_name, index=False)

In [72]:
print(f"Cleaned dataset saved as: {imputed_file_name}")

Cleaned dataset saved as: cars_data_CROSS_IMPUTED.csv
